# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.
  - **Director**: The director of the movie.
  - **Stars**: The main stars of the movie.
  - **Votes**: The number of votes the movie received.
  - **Gross ($M)**: The gross earnings of the movie in millions of USD.

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

In [43]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

url="https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,"
response= requests.get(url,headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"})
soup=BeautifulSoup(response.content,"html.parser")




In [42]:
films=soup.find_all("li",attrs={"class":"ipc-metadata-list-summary-item"})
film_chart = {}
key = 0
for film in films:
    film_titles=(film.find_all("h3", attrs = {"class":"ipc-title__text"})[0].get_text())
    film_num=film_titles.split(".")[0]
    film_title=film_titles.split(".")[1::]
    film_year=(film.find_all("span", attrs = {"class":"sc-b189961a-8 kLaxqf dli-title-metadata-item"})[0].get_text())
    film_ratings=(film.find_all("span", attrs = {"class":"ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating"})[0].get_text())
    film_rating=film_ratings.split()[0]
    film_runtime=(film.find_all("span", attrs = {"class":"sc-b189961a-8 kLaxqf dli-title-metadata-item"})[1].get_text())
    film_desc=(film.find_all("div", attrs = {"class":"ipc-html-content-inner-div"})[0].get_text())
    try:
        film_score=(film.find_all("span", attrs = {"class":"sc-b0901df4-0 bcQdDJ metacritic-score-box"})[0].get_text())
    except:
        film_score=0
#Genre, Director, Stars and Gross were discarded as cathegory for site renewal.

    key +=1
    film_chart[key] = {"Movie Nr":film_num,
                            "Title":film_title,
                            "Year":film_year,
                            "Rating":film_rating,
                            "Runtime":film_runtime,
                            "Description":film_desc,
                            "Vote":film_score}

df = pd.DataFrame.from_dict(film_chart).T
df

def scrape_imdb():
    display(df)
scrape_imdb()

,Movie Nr,Title,Year,Rating,Runtime,Description,Vote
1,1,[ O Silêncio dos Inocentes],1991,8.6,1h 58m,A young F.B.I. cadet must receive the help of ...,86
2,2,[ Tudo Bons Rapazes],1990,8.7,2h 25m,The story of Henry Hill and his life in the ma...,92
3,3,[ Exterminador Implacável 2 - O Dia do Julgame...,1991,8.6,2h 17m,"A cyborg, identical to the one who failed to k...",75
4,4,[ Imperdoável],1992,8.2,2h 10m,Retired Old West gunslinger William Munny relu...,85
5,5,[ Cães Danados],1992,8.3,1h 39m,When a simple jewelry heist goes horribly wron...,81
6,6,[ O Padrinho: Parte III],1990,7.6,2h 42m,"Follows Michael Corleone, now in his 60s, as h...",60
7,7,[ Desafio Total],1990,7.5,1h 53m,When a man goes in to have virtual vacation me...,60
8,8,[ Mulheres do Sul],1991,7.7,2h 10m,"Evelyn, an ordinary housewife, visits a nursin...",64
9,9,[ O Meu Primo Vinny],1992,7.6,2h,Two New Yorkers accused of murder in rural Ala...,68
10,10,[ Danças com Lobos],1990,8.0,3h 1m,"Lieutenant John Dunbar, assigned to a remote w...",72


## BONUS

The search results span multiple pages, housing a total of 631 movies in our example with each page displaying 50 movies at most. To scrape data seamlessly from all pages, you'll need to dive deep into the structure of the URLs generated with each "Next" click.

Take a close look at the following URLs:
- First page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,
  ```
- Second page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt
  ```
- Third page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt
  ```

You should notice a pattern. There is a `start` parameter incrementing by 50 with each page, paired with a constant `ref_` parameter holding the value "adv_nxt".

Modify your script so it's capable of iterating over all available pages to fetch data on all the 631 movies (631 is the total number of movies in the proposed example).

In [ ]:
# Your solution goes here